Importing the Dependencies

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

Data Collection & Pre-Processing

In [2]:
raw_mail_data = pd.read_csv('mail_data.csv')

In [3]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [4]:
# replace the null values with a null string
mail_data = raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [5]:
mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
mail_data.shape

(5572, 2)

Label Encoding

In [8]:
# label spam mail as 0;  ham mail as 1;
#.loc[condition, column] = new_value


mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

spam  -  0

ham  -  1

In [9]:
# separating the data as texts and label

X = mail_data['Message']

Y = mail_data['Category']

In [10]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [11]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


Splitting the data into training data & test data

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [13]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


Feature Extraction

In [14]:
# -> feature extraction
# TF (Term Frequency): How often a word appears in a document.
# IDF (Inverse Document Frequency): How unique or rare a word is across all documents.
# Together, TF-IDF helps to give higher weight to important words and reduce weight for common, less informative ones.

# -> min_df=1
# This sets the minimum number of documents a word must appear in to be included in the vocabulary.
# min_df=1 means: include all words that appear in at least 1 document (i.e., everything).
# You can set min_df=2 or higher to filter out rare words.

# -> stop_words='English'
# This tells the vectorizer to ignore common English stop words, such as:
# "the", "and", "is", "in", "at", etc.
# ✅ Helps reduce noise and size of the feature space.

# -> lowercase=True
# Converts all text to lowercase before processing, so "Spam" and "spam" are treated the same.


In [15]:
feature_extraction = TfidfVectorizer(min_df=1,stop_words='english',lowercase=True)

In [16]:
#fit: Learns the vocabulary (words) and their IDF scores from X_train.

#transform: Converts the X_train text into TF-IDF numeric vectors based on that vocabulary.

#imp
#Use .transform() on test data so it uses the same vocabulary and IDF learned from training data.

In [17]:
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

Training the Model

Logistic Regression

In [18]:
model = LogisticRegression()

In [19]:
# training the Logistic Regression model with the training data
model.fit(X_train_features, Y_train)

LogisticRegression()

Evaluating the trained model

In [20]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_features)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [21]:
print('Accuracy on training data : ', accuracy_on_training_data)

Accuracy on training data :  0.9676912721561588


In [22]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [23]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.9668161434977578


Building a Predictive System

In [24]:
# Building a Predictive System

def check_mail(input_mail):
    input_mail_features = feature_extraction.transform([input_mail])
    prediction = model.predict(input_mail_features)
    if(prediction[0]==1):
        print("It is not a spam mail!")
    else:
        print("It is a spam mail")

In [25]:
check_mail("Hello, Harsh! How have you been?")

It is not a spam mail!


In [26]:
check_mail("Congratulations!!! You won a FREE lottery. Click here now!")

It is a spam mail
